In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import datetime
import re
import numpy as np
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import seaborn as sns
import numpy as np
from itertools import groupby
!pip install folium

## Read Data

In [2]:
orig_exonerations_df = pd.read_csv(r"C:\Users\cindy\OneDrive\Documents\DA15\Python\Projects\exonerations\data\nre_export_2025-07-24.csv")

In [3]:
orig_exonerations_df.head()

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,NaN,14.2,...,No,No,No,No,No,No,No,No,No,NaN
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,NaN,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,NaN
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,NaN,30.5,...,No,No,No,No,No,No,No,No,No,NaN
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,NaN
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,NaN,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006


## Cleaning

In [4]:
# Filling in empty cells with "Not Provided"
exonerations_df = orig_exonerations_df.fillna('Not Provided')

In [5]:
# Seeing the types of each column
print(exonerations_df.dtypes)

Name                                       object
State                                      object
County                                     object
Age at Crime Yrs                           object
Sex                                        object
Race                                       object
Crime                                      object
Sentence                                   object
Term of Years                              object
Years lost                                float64
Pre-1989                                   object
Date of Exon                               object
Date of 1st Convic                         object
Date of Crime Month                        object
Date of Crime Day                          object
Date of Crime Year                          int64
Date of Release                            object
Posting Date                               object
Case Characteristics                       object
DNA                                        object


In [6]:
exonerations_df.columns

Index(['Name', 'State', 'County', 'Age at Crime Yrs', 'Sex', 'Race', 'Crime',
       'Sentence', 'Term of Years', 'Years lost', 'Pre-1989', 'Date of Exon',
       'Date of 1st Convic', 'Date of Crime Month', 'Date of Crime Day',
       'Date of Crime Year ', 'Date of Release', 'Posting Date',
       'Case Characteristics', 'DNA', 'False Confession ',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense ',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering', 'Recantation',
       'Year of recantation'],
      dtype='object')

In [7]:
exonerations_df.head()

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006


In [8]:
exonerations_df['Race'] = exonerations_df['Race'].replace("Don't Know", "Unknown")

In [9]:
exonerations_df = exonerations_df.rename(columns={'Years lost': 'Years Lost'})

In [10]:
exonerations_df = exonerations_df.rename(columns={'Inadequate Legal Defense ': 'Inadequate Legal Defense'})

In [11]:
exonerations_df = exonerations_df.rename(columns={'False Confession ': 'False Confession'})

In [12]:
exonerations_df = exonerations_df.rename(columns={'Date of Crime Year ': 'Date of Crime Year'})

In [13]:
exonerations_df['Age at Crime Yrs'] = exonerations_df['Age at Crime Yrs'].replace("Not Provided", 0)

C:\Users\cindy\AppData\Local\Temp\ipykernel_48344\4127705870.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  exonerations_df['Age at Crime Yrs'] = exonerations_df['Age at Crime Yrs'].replace("Not Provided", 0)


In [14]:
# to download the new data frame
exonerations_df.to_csv('exonerations_df.csv', index=False)

## Exploring Data

In [15]:
# Organizing by State
exonerations_df.sort_values(by = 'State')

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
3322,"Sealie, Frank",Alabama,Jefferson,23.0,Male,Black,Murder,Life without parole,Not Provided,0.8,...,No,No,No,No,No,No,No,No,Yes,2014
1785,"Hunt, H. Guy",Alabama,Montgomery,54.0,Male,White,Official Misconduct,Probation,Not Provided,0.0,...,No,No,No,No,No,No,No,No,No,Not Provided
1320,"Geeslin, Mark",Alabama,Madison,23.0,Male,White,Sexual Assault,Life,Life,2.9,...,No,No,No,No,No,No,No,No,No,Not Provided
1736,"Holsomback, John",Alabama,Shelby,33.0,Male,White,Child Sex Abuse,Term of Years,25 years,10.1,...,No,No,No,No,No,No,No,No,No,Not Provided
1723,"Holemon, Jeffrey",Alabama,Tuscaloosa,23.0,Male,White,Sexual Assault,Life,Not Provided,10.6,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3266,"Sanders, Derrick",Wisconsin,Milwaukee,21.0,Male,Black,Murder,Term of Years,21 to life,25.0,...,No,No,No,No,No,No,No,No,Yes,1996
4077,"Willoughby, Troy",Wyoming,Sublette,20.0,Male,White,Murder,Life,Not Provided,2.0,...,No,No,No,No,No,No,Yes,No,No,Not Provided
1893,"Johnson, Andrew",Wyoming,Laramie,39.0,Male,Black,Sexual Assault,Life,Not Provided,23.6,...,No,No,No,No,No,No,No,No,No,Not Provided
1029,"Drennen, Gabriel",Wyoming,Fremont,36.0,Male,White,Murder,Life,Not Provided,2.9,...,No,Yes,Yes,No,No,No,Yes,No,No,Not Provided


In [16]:
# Top 5 States with the most exonerations
top5_states = exonerations_df['State'].value_counts().head()

In [17]:
top5_states

State
Illinois      608
Texas         515
New York      428
California    336
Michigan      213
Name: count, dtype: int64

In [18]:
# Top 5 Crimes with the most exonerations
top5_crime = exonerations_df['Crime'].value_counts().head()

In [19]:
top5_crime

Crime
Murder                     1698
Drug Possession or Sale     645
Sexual Assault              414
Child Sex Abuse             345
Robbery                     257
Name: count, dtype: int64

In [20]:
# Top 5 Races with the most exonerations
top5_race = exonerations_df['Race'].value_counts().head()

In [21]:
top5_race

Race
Black       2114
White       1411
Hispanic     499
Unknown       75
Asian         34
Name: count, dtype: int64

In [22]:
# What has been the highest amount of "Years lost" 
high_yearslost = exonerations_df['Years Lost'].max()

In [23]:
high_yearslost

48.1

In [24]:
# The average amount of "Years lost"
avg_yearslost = exonerations_df['Years Lost'].mean()

In [25]:
avg_yearslost

np.float64(8.754936648338512)

In [26]:
# What are the most common Ages getting convicted and exonerated
top5_ages = exonerations_df['Age at Crime Yrs'].value_counts().head()

In [27]:
top5_ages

Age at Crime Yrs
20.0    244
19.0    226
21.0    214
18.0    208
22.0    204
Name: count, dtype: int64

In [28]:
# How many men or women have been exonerated
gender_df = exonerations_df['Sex'].value_counts()

In [29]:
gender_df

Sex
Male      3849
Female     334
Name: count, dtype: int64

In [30]:
# New df showing Exonerations with Murder Charge and Inadequate Legal Defense
exonerations_df[(exonerations_df['Inadequate Legal Defense'] == "Yes") & (exonerations_df['Crime'] == "Murder")]

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
6,"Abramowski, Jeffrey",Florida,Brevard,40.0,Male,White,Murder,Life,Life,18.8,...,No,No,No,No,No,No,No,No,No,Not Provided
19,"Adams, Laurence",Massachusetts,Suffolk,19.0,Male,Black,Murder,Death,Not Provided,30.1,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,2003
26,"Addison, Ronald",Maryland,Baltimore City,19.0,Male,Black,Murder,Term of Years,30 years,7.7,...,No,No,No,No,No,No,Yes,No,Yes,2003
29,"Aguirre-Jarquin, Clemente",Florida,Seminole,24.0,Male,Hispanic,Murder,Death,Not Provided,12.7,...,No,No,No,No,No,No,No,No,Yes,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,"Wright, David",Illinois,Cook,17.0,Male,Black,Murder,Life without parole,Not Provided,25.9,...,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,2017
4154,"Yanez, Jose",Illinois,Rock Island,28.0,Male,Hispanic,Murder,Term of Years,15 years,2.4,...,No,No,No,No,No,No,No,No,No,Not Provided
4156,"Yarbough, Anthony",New York,Kings,18.0,Male,Black,Murder,Term of Years,75 to life,20.0,...,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,2005
4157,"Yarris, Nicholas",Pennsylvania,Delaware,20.0,Male,White,Murder,Death,Not Provided,21.2,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1983


In [31]:
exonerations_df.columns

Index(['Name', 'State', 'County', 'Age at Crime Yrs', 'Sex', 'Race', 'Crime',
       'Sentence', 'Term of Years', 'Years Lost', 'Pre-1989', 'Date of Exon',
       'Date of 1st Convic', 'Date of Crime Month', 'Date of Crime Day',
       'Date of Crime Year', 'Date of Release', 'Posting Date',
       'Case Characteristics', 'DNA', 'False Confession',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering', 'Recantation',
       'Year of recantation'],
      dtype='object')

In [32]:
# Trying to see which factors are more common when it comes to exonerations
factors_df = exonerations_df[['DNA', 'False Confession',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?']].apply(pd.Series.value_counts)

In [33]:
factors_df
# Seems like Perjury or False Accusation has the most "Yes"

,DNA,False Confession,False or Misleading Forensic Evidence?,Inadequate Legal Defense,Mistaken Witness ID,Official Misconduct,Perjury or False Accusation?
No,3554,3648,3074,3112,2982,1775,1585
Yes,629,535,1109,1071,1201,2408,2598


In [34]:
# Counting all the yes for each facto
yesfactor_count = factors_df.loc["Yes"]

In [35]:
yesfactor_count

DNA                                        629
False Confession                           535
False or Misleading Forensic Evidence?    1109
Inadequate Legal Defense                  1071
Mistaken Witness ID                       1201
Official Misconduct                       2408
Perjury or False Accusation?              2598
Name: Yes, dtype: int64

In [36]:
exonerations_df[(exonerations_df['Official Misconduct'] == "Yes")]

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
7,"Abrego, Eruby",Illinois,Cook,20.0,Male,Hispanic,Murder,Term of Years,90 years,17.8,...,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,2012
9,"Achberger, Terry E.",Ohio,Summit,28.0,Male,White,Murder,Life,Not Provided,5.4,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,"Zimmer, Walter",Ohio,Cuyahoga,40.0,Male,White,Manslaughter,Term of Years,50 years,12.7,...,No,Yes,Yes,Yes,No,No,No,No,No,Not Provided
4179,"Zimmerman, Isidore",New York,New York,19.0,Male,White,Murder,Death,Not Provided,23.8,...,No,No,No,No,No,No,No,No,No,Not Provided
4180,"Zinkiewicz, Tyrone",Ohio,Montgomery,38.0,Male,White,Other Nonviolent Felony,Term of Years,5 to 15 years,3.1,...,No,No,No,No,No,No,Yes,No,No,Not Provided
4181,"Zomber, Michael",Fed-PA,(Eastern),53.0,Male,White,Fraud,Term of Years,2 years and 6 months,4.5,...,No,No,No,No,No,No,Yes,No,No,Not Provided


In [37]:
om_count = exonerations_df[['Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering']].apply(pd.Series.value_counts)

In [38]:
om_count

,Child Welfare Worker Misconduct,Exculpatory Evidence Withheld,Forensic Analyst Misconduct,Knowingly Permitting Perjury,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering
No,4130,2273,4079,3864,3925,2304,2304,3534,2543,3973,2983,3316
Yes,53,1910,104,319,258,1879,1879,649,1640,210,1200,867


In [39]:
yesmiscon_count = om_count.loc["Yes"]

In [40]:
yesmiscon_count

Child Welfare Worker Misconduct         53
Exculpatory Evidence Withheld         1910
Forensic Analyst Misconduct            104
Knowingly Permitting Perjury           319
Misconduct in Interrogation            258
Misconduct in Interrogation.1         1879
Misconduct that is not withholding    1879
OM - Perjury                           649
Police Misconduct                     1640
Prosecutor Lied in Court               210
Prosecutor Misconduct                 1200
Witness Tampering                      867
Name: Yes, dtype: int64

In [41]:
# Seeing how race takes place when it comes to false accusation 
exonerations_df[(exonerations_df['Knowingly Permitting Perjury'] == "Yes") & (exonerations_df['Race'] == "Black")]

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
18,"Adams, Kenneth",Illinois,Cook,22.0,Male,Black,Murder,Term of Years,75 years,17.7,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1994
19,"Adams, Laurence",Massachusetts,Suffolk,19.0,Male,Black,Murder,Death,Not Provided,30.1,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,2003
21,"Adams, Reginald",Louisiana,Orleans,26.0,Male,Black,Murder,Life without parole,Not Provided,30.7,...,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,Not Provided
23,"Adams, Sean",Connecticut,New Haven,21.0,Male,Black,Murder,Life,Not Provided,13.6,...,No,Yes,Yes,No,No,No,Yes,No,No,Not Provided
55,"Allen, Dennis",Texas,Dallas,36.0,Male,Black,Murder,Life,Not Provided,14.2,...,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097,"Wilson, Theophalis",Pennsylvania,Philadelphia,17.0,Male,Black,Murder,Life without parole,Not Provided,26.5,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,2011
4144,"Wright, Isaac Jr.",New Jersey,Somerset,28.0,Male,Black,Drug Possession or Sale,Life,Not Provided,5.6,...,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,1996
4148,"Wright, William Jr.",North Carolina,New Hanover,18.0,Male,Black,Assault,Term of Years,29 years,7.9,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,1975
4158,"Yates, Michael",Indiana,Howard,25.0,Male,Black,"Attempt, Violent",Term of Years,20 years,0.8,...,No,Yes,Yes,No,Yes,No,Yes,Yes,No,Not Provided


In [42]:
exonerations_df['Date of Crime Year'].drop_duplicates().sort_values()

6          0
340     1812
279     1850
2343    1873
3129    1876
        ... 
1225    2019
898     2020
1117    2021
308     2022
1730    2023
Name: Date of Crime Year, Length: 139, dtype: int64

In [43]:
## Most of the defendants had sentencing amounts that weren't life
exonerations_df[['Sentence']].value_counts()

Sentence           
Term of Years          2580
Life                    654
Life without parole     310
Death                   244
Probation               208
Not Sentenced           160
Unknown                  15
Community Service         7
Fine                      5
Name: count, dtype: int64

In [44]:
exonerations_df[['State']].value_counts()

State     
Illinois      608
Texas         515
New York      428
California    336
Michigan      213
             ... 
Fed-ND          1
Fed-NM          1
Fed-SD          1
Fed-NV          1
Fed-MT          1
Name: count, Length: 89, dtype: int64

## Captstone Analysis

In [45]:
# Introducing Project - talkion about dataset

In [46]:
exonerations_df.columns

Index(['Name', 'State', 'County', 'Age at Crime Yrs', 'Sex', 'Race', 'Crime',
       'Sentence', 'Term of Years', 'Years Lost', 'Pre-1989', 'Date of Exon',
       'Date of 1st Convic', 'Date of Crime Month', 'Date of Crime Day',
       'Date of Crime Year', 'Date of Release', 'Posting Date',
       'Case Characteristics', 'DNA', 'False Confession',
       'False or Misleading Forensic Evidence?', 'Inadequate Legal Defense',
       'Mistaken Witness ID', 'Official Misconduct',
       'Perjury or False Accusation?', 'Child Welfare Worker Misconduct',
       'Exculpatory Evidence Withheld', 'Forensic Analyst Misconduct',
       'Knowingly Permitting Perjury', 'Misconduct in Interrogation',
       'Misconduct in Interrogation.1', 'Misconduct that is not withholding',
       'OM - Perjury', 'Police Misconduct', 'Prosecutor Lied in Court',
       'Prosecutor Misconduct', 'Witness Tampering', 'Recantation',
       'Year of recantation'],
      dtype='object')

In [47]:
#Listing out my data
exonerations_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4178,"Zimmerman, Evan",Wisconsin,Eau Claire,53.0,Male,White,Murder,Life,Not Provided,3.1,...,No,No,No,No,No,No,No,No,No,Not Provided
4179,"Zimmerman, Isidore",New York,New York,19.0,Male,White,Murder,Death,Not Provided,23.8,...,No,No,No,No,No,No,No,No,No,Not Provided
4180,"Zinkiewicz, Tyrone",Ohio,Montgomery,38.0,Male,White,Other Nonviolent Felony,Term of Years,5 to 15 years,3.1,...,No,No,No,No,No,No,Yes,No,No,Not Provided
4181,"Zomber, Michael",Fed-PA,(Eastern),53.0,Male,White,Fraud,Term of Years,2 years and 6 months,4.5,...,No,No,No,No,No,No,Yes,No,No,Not Provided


In [48]:
# Then go into the Pre-1989 and case
exonerations_df[['Pre-1989']].value_counts()

Pre-1989
No          3719
Yes          464
Name: count, dtype: int64

In [49]:
pre1989_df = exonerations_df[(exonerations_df['Pre-1989'] == "Yes")]

In [50]:
pre1989_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
9,"Achberger, Terry E.",Ohio,Summit,28.0,Male,White,Murder,Life,Not Provided,5.4,...,No,No,No,No,No,No,No,No,No,Not Provided
47,"Alford, John",North Carolina,Mecklenburg,22.0,Male,Black,Murder,Death,Not Provided,1.2,...,No,No,No,No,No,No,No,No,No,Not Provided
99,"Anderson, Robert",Michigan,Kent,0.0,Male,Black,Theft,Term of Years,4 years,2.1,...,No,No,No,No,No,No,No,No,No,Not Provided
105,"Andrews, Herbert",Massachusetts,Suffolk,0.0,Male,White,Forgery,Term of Years,1.17 years,0.3,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,"Woodward, Walter",Florida,Broward,32.0,Male,Black,Murder,Death,Not Provided,8.8,...,No,No,No,No,No,No,No,No,No,Not Provided
4160,"Yelder, George",Alabama,Lowndes,55.0,Male,Black,Murder,Life,Not Provided,0.2,...,No,No,No,No,No,No,No,No,No,Not Provided
4170,"Zajicek, Herman",Illinois,Cook,39.0,Male,White,Murder,Death,Not Provided,9.5,...,No,No,No,No,No,No,No,No,No,Not Provided
4171,"Zambino, Luigi",New Jersey,Mercer,0.0,Male,White,Forgery,Term of Years,6 years,3.4,...,No,No,No,No,No,No,No,No,No,Not Provided


In [51]:
post1989_df = exonerations_df[(exonerations_df['Pre-1989'] == "No")]

In [52]:
post1989_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
5,"Abney, Quentin",New York,New York,32.0,Male,Black,Robbery,Term of Years,20 to Life,5.8,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4177,"Zimmer, Walter",Ohio,Cuyahoga,40.0,Male,White,Manslaughter,Term of Years,50 years,12.7,...,No,Yes,Yes,Yes,No,No,No,No,No,Not Provided
4178,"Zimmerman, Evan",Wisconsin,Eau Claire,53.0,Male,White,Murder,Life,Not Provided,3.1,...,No,No,No,No,No,No,No,No,No,Not Provided
4180,"Zinkiewicz, Tyrone",Ohio,Montgomery,38.0,Male,White,Other Nonviolent Felony,Term of Years,5 to 15 years,3.1,...,No,No,No,No,No,No,Yes,No,No,Not Provided
4181,"Zomber, Michael",Fed-PA,(Eastern),53.0,Male,White,Fraud,Term of Years,2 years and 6 months,4.5,...,No,No,No,No,No,No,Yes,No,No,Not Provided


In [53]:
# See how many exonerations happened since 1989
DNAexon_df = exonerations_df[(exonerations_df['DNA'] == "Yes")]

In [54]:
DNAexon_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
6,"Abramowski, Jeffrey",Florida,Brevard,40.0,Male,White,Murder,Life,Life,18.8,...,No,No,No,No,No,No,No,No,No,Not Provided
12,"Adams, Darryl",Texas,Dallas,25.0,Male,Black,Sexual Assault,Term of Years,25 years,21.9,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4164,"Young, Dan Jr.",Illinois,Cook,30.0,Male,Black,Murder,Life without parole,Not Provided,10.4,...,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4167,"Youngblood, Larry",Arizona,Pima,30.0,Male,Black,Child Sex Abuse,Term of Years,10 years and 6 months,15.2,...,No,No,No,No,No,No,No,No,No,Not Provided
4168,"Yurtman, Ersan",Texas,Bexar,18.0,Male,Hispanic,Child Sex Abuse,Term of Years,30 years,26.4,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1994
4177,"Zimmer, Walter",Ohio,Cuyahoga,40.0,Male,White,Manslaughter,Term of Years,50 years,12.7,...,No,Yes,Yes,Yes,No,No,No,No,No,Not Provided


In [78]:
raceDNA_count = DNAexon_df[['Race']].value_counts()

In [80]:
raceDNA_count

Race           
Black              349
White              209
Hispanic            62
Native American      4
Other                3
Asian                2
Name: count, dtype: int64

In [56]:
# Then put percentage of DNA exonerations since 1989 from the whole dataset which is 15.04%

In [57]:
preDNA_om_df = exonerations_df[(exonerations_df['Pre-1989'] == "Yes") & (exonerations_df['Official Misconduct'] == "Yes")]

In [58]:
preDNA_om_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
9,"Achberger, Terry E.",Ohio,Summit,28.0,Male,White,Murder,Life,Not Provided,5.4,...,No,No,No,No,No,No,No,No,No,Not Provided
132,"Artis, John",New Jersey,Passaic,19.0,Male,Black,Murder,Term of Years,15 years to life,14.6,...,No,No,No,No,No,No,No,No,No,Not Provided
152,"Avila, Edward",California,Los Angeles,29.0,Male,Hispanic,Robbery,Term of Years,7 years to life,0.3,...,No,No,No,No,No,No,No,No,No,Not Provided
199,"Banks, Jerry",Georgia,Henry,23.0,Male,Black,Murder,Death,Not Provided,5.9,...,No,No,No,No,No,No,No,No,No,Not Provided
297,"Bilger, George",Pennsylvania,Philadelphia,19.0,Male,White,Murder,Death,Not Provided,2.5,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4064,"Williamson, Jack",Florida,Broward,19.0,Male,Black,Murder,Death,Not Provided,8.8,...,No,No,No,No,No,No,No,No,No,Not Provided
4091,"Wilson, Horace",New Jersey,Mercer,37.0,Male,Black,Murder,Death,Not Provided,2.9,...,No,No,No,No,No,No,No,No,No,Not Provided
4126,"Woodward, Walter",Florida,Broward,32.0,Male,Black,Murder,Death,Not Provided,8.8,...,No,No,No,No,No,No,No,No,No,Not Provided
4170,"Zajicek, Herman",Illinois,Cook,39.0,Male,White,Murder,Death,Not Provided,9.5,...,No,No,No,No,No,No,No,No,No,Not Provided


In [59]:
preDNA_om_df['Crime'].value_counts()

Crime
Murder                     113
Sexual Assault              13
Robbery                     12
Other Nonviolent Felony      7
Manslaughter                 4
Child Sex Abuse              4
Attempted Murder             2
Theft                        2
Assault                      1
Name: count, dtype: int64

In [60]:
postDNA_om_df = exonerations_df[(exonerations_df['Pre-1989'] == "No") & (exonerations_df['Official Misconduct'] == "Yes")]

In [61]:
postDNA_om_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
1,"Abbott, Cinque",Illinois,Cook,19.0,Male,Black,Drug Possession or Sale,Probation,Not Provided,0.0,...,No,Yes,Yes,No,Yes,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
7,"Abrego, Eruby",Illinois,Cook,20.0,Male,Hispanic,Murder,Term of Years,90 years,17.8,...,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,2012
10,"Adams, Anthony",California,Los Angeles,26.0,Male,Hispanic,Manslaughter,Term of Years,12 years,5.6,...,No,Yes,Yes,No,Yes,No,No,Yes,Yes,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4174,"Zapata, Rodrigo",Texas,Webb,25.0,Male,Hispanic,Robbery,Term of Years,25 years,1.9,...,No,No,No,No,No,No,Yes,No,No,Not Provided
4177,"Zimmer, Walter",Ohio,Cuyahoga,40.0,Male,White,Manslaughter,Term of Years,50 years,12.7,...,No,Yes,Yes,Yes,No,No,No,No,No,Not Provided
4180,"Zinkiewicz, Tyrone",Ohio,Montgomery,38.0,Male,White,Other Nonviolent Felony,Term of Years,5 to 15 years,3.1,...,No,No,No,No,No,No,Yes,No,No,Not Provided
4181,"Zomber, Michael",Fed-PA,(Eastern),53.0,Male,White,Fraud,Term of Years,2 years and 6 months,4.5,...,No,No,No,No,No,No,Yes,No,No,Not Provided


In [62]:
postDNA_om_df['Crime'].value_counts()

Crime
Murder                           1070
Drug Possession or Sale           419
Sexual Assault                    162
Child Sex Abuse                   144
Robbery                            79
Assault                            71
Attempted Murder                   52
Weapon Possession or Sale          51
Manslaughter                       38
Fraud                              36
Burglary/Unlawful Entry            14
Arson                              11
Kidnapping                         11
Attempt, Violent                   10
Other Violent Felony                8
Other Nonviolent Felony             7
Traffic Offense                     7
Other Nonviolent Misdemeanor        6
Accessory to Murder                 6
Theft                               5
Immigration                         4
Supporting Terrorism                4
Child Abuse                         4
Other Violent Misdemeanor           4
Tax Evasion/Fraud                   3
Perjury                             3
Briber

In [63]:
preDNA_om_df['Race'].value_counts()

Race
White       92
Black       52
Unknown      9
Hispanic     3
Asian        2
Name: count, dtype: int64

In [64]:
postDNA_om_df['Race'].value_counts()

Race
Black              1307
White               599
Hispanic            302
Asian                17
Native American      13
Other                 7
Unknown               5
Name: count, dtype: int64

In [65]:
DNAexon_df['Race'].value_counts()

Race
Black              349
White              209
Hispanic            62
Native American      4
Other                3
Asian                2
Name: count, dtype: int64

In [66]:
blackmurd_df = exonerations_df[(exonerations_df['Crime'] == "Murder") & (exonerations_df['Race'] == "Black") & (exonerations_df['Sentence'])]

In [67]:
blackmurd_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
14,"Adams, Don Ray",Pennsylvania,Philadelphia,32.0,Male,Black,Murder,Life,Not Provided,18.8,...,No,Yes,Yes,No,Yes,No,No,Yes,Yes,2007
18,"Adams, Kenneth",Illinois,Cook,22.0,Male,Black,Murder,Term of Years,75 years,17.7,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1994
19,"Adams, Laurence",Massachusetts,Suffolk,19.0,Male,Black,Murder,Death,Not Provided,30.1,...,No,Yes,Yes,No,No,No,Yes,Yes,Yes,2003
21,"Adams, Reginald",Louisiana,Orleans,26.0,Male,Black,Murder,Life without parole,Not Provided,30.7,...,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,Not Provided
23,"Adams, Sean",Connecticut,New Haven,21.0,Male,Black,Murder,Life,Not Provided,13.6,...,No,Yes,Yes,No,No,No,Yes,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4141,"Wright, David",Illinois,Cook,17.0,Male,Black,Murder,Life without parole,Not Provided,25.9,...,Yes,Yes,Yes,Yes,Yes,No,No,No,Yes,2017
4156,"Yarbough, Anthony",New York,Kings,18.0,Male,Black,Murder,Term of Years,75 to life,20.0,...,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,2005
4160,"Yelder, George",Alabama,Lowndes,55.0,Male,Black,Murder,Life,Not Provided,0.2,...,No,No,No,No,No,No,No,No,No,Not Provided
4164,"Young, Dan Jr.",Illinois,Cook,30.0,Male,Black,Murder,Life without parole,Not Provided,10.4,...,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided


In [68]:
blackmurd_df[['Sentence']].value_counts()

Sentence           
Term of Years          366
Life                   214
Life without parole    181
Death                  116
Not Sentenced            5
Name: count, dtype: int64

In [69]:
whitemurd_df = exonerations_df[(exonerations_df['Crime'] == "Murder") & (exonerations_df['Race'] == "White") & (exonerations_df['Sentence'])]

In [70]:
whitemurd_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
2,"Abbott, Stearns Kendall",Massachusetts,Middlesex,40.0,Male,White,Murder,Death,Not Provided,30.5,...,No,No,No,No,No,No,No,No,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
6,"Abramowski, Jeffrey",Florida,Brevard,40.0,Male,White,Murder,Life,Life,18.8,...,No,No,No,No,No,No,No,No,No,Not Provided
9,"Achberger, Terry E.",Ohio,Summit,28.0,Male,White,Murder,Life,Not Provided,5.4,...,No,No,No,No,No,No,No,No,No,Not Provided
16,"Adams, Johnathan",Georgia,Carroll,12.0,Male,White,Murder,Term of Years,1 year,0.7,...,Yes,Yes,Yes,No,Yes,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4157,"Yarris, Nicholas",Pennsylvania,Delaware,20.0,Male,White,Murder,Death,Not Provided,21.2,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1983
4170,"Zajicek, Herman",Illinois,Cook,39.0,Male,White,Murder,Death,Not Provided,9.5,...,No,No,No,No,No,No,No,No,No,Not Provided
4178,"Zimmerman, Evan",Wisconsin,Eau Claire,53.0,Male,White,Murder,Life,Not Provided,3.1,...,No,No,No,No,No,No,No,No,No,Not Provided
4179,"Zimmerman, Isidore",New York,New York,19.0,Male,White,Murder,Death,Not Provided,23.8,...,No,No,No,No,No,No,No,No,No,Not Provided


In [71]:
whitemurd_df[['Sentence']].value_counts()

Sentence           
Life                   198
Term of Years          176
Death                   97
Life without parole     77
Not Sentenced           12
Name: count, dtype: int64

In [72]:
top5_crime

Crime
Murder                     1698
Drug Possession or Sale     645
Sexual Assault              414
Child Sex Abuse             345
Robbery                     257
Name: count, dtype: int64

In [73]:
filtered = exonerations_df[
    exonerations_df['Race'].isin(['Black', 'White']) &
    exonerations_df['Crime'].isin(['Murder']) &
    exonerations_df['Sentence']
]

In [74]:
filtered.groupby(['Race', 'Crime', 'Sentence']).size().reset_index(name='Count')

,Race,Crime,Sentence,Count
0,Black,Murder,Death,116
1,Black,Murder,Life,214
2,Black,Murder,Life without parole,181
3,Black,Murder,Not Sentenced,5
4,Black,Murder,Term of Years,366
5,White,Murder,Death,97
6,White,Murder,Life,198
7,White,Murder,Life without parole,77
8,White,Murder,Not Sentenced,12
9,White,Murder,Term of Years,176


In [75]:
exonerations_df[['Race']].value_counts()

Race           
Black              2114
White              1411
Hispanic            499
Unknown              75
Asian                34
Native American      28
Other                22
Name: count, dtype: int64

In [76]:
top5_race

Race
Black       2114
White       1411
Hispanic     499
Unknown       75
Asian         34
Name: count, dtype: int64

In [77]:
## black: 50.5378915 white: 33.7317715

In [82]:
DNAexon_df

,Name,State,County,Age at Crime Yrs,Sex,Race,Crime,Sentence,Term of Years,Years Lost,...,Misconduct in Interrogation,Misconduct in Interrogation.1,Misconduct that is not withholding,OM - Perjury,Police Misconduct,Prosecutor Lied in Court,Prosecutor Misconduct,Witness Tampering,Recantation,Year of recantation
0,"Abbitt, Joseph",North Carolina,Forsyth,31.0,Male,Black,Child Sex Abuse,Life,Not Provided,14.2,...,No,No,No,No,No,No,No,No,No,Not Provided
3,"Abdal, Warith Habib",New York,Erie,43.0,Male,Black,Sexual Assault,Term of Years,20 to Life,16.2,...,No,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4,"Abernathy, Christopher",Illinois,Cook,17.0,Male,White,Murder,Life without parole,Not Provided,28.1,...,Yes,Yes,Yes,No,Yes,No,No,No,Yes,2006
6,"Abramowski, Jeffrey",Florida,Brevard,40.0,Male,White,Murder,Life,Life,18.8,...,No,No,No,No,No,No,No,No,No,Not Provided
12,"Adams, Darryl",Texas,Dallas,25.0,Male,Black,Sexual Assault,Term of Years,25 years,21.9,...,No,No,No,No,No,No,No,No,No,Not Provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4164,"Young, Dan Jr.",Illinois,Cook,30.0,Male,Black,Murder,Life without parole,Not Provided,10.4,...,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Not Provided
4167,"Youngblood, Larry",Arizona,Pima,30.0,Male,Black,Child Sex Abuse,Term of Years,10 years and 6 months,15.2,...,No,No,No,No,No,No,No,No,No,Not Provided
4168,"Yurtman, Ersan",Texas,Bexar,18.0,Male,Hispanic,Child Sex Abuse,Term of Years,30 years,26.4,...,No,Yes,Yes,No,Yes,No,Yes,Yes,Yes,1994
4177,"Zimmer, Walter",Ohio,Cuyahoga,40.0,Male,White,Manslaughter,Term of Years,50 years,12.7,...,No,Yes,Yes,Yes,No,No,No,No,No,Not Provided


In [85]:
race_DNAOM_df = DNAexon_df[(exonerations_df['Official Misconduct'] == "Yes") & (exonerations_df['Race'] == "Black")]

C:\Users\cindy\AppData\Local\Temp\ipykernel_48344\2288105391.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  race_DNAOM_df = DNAexon_df[(exonerations_df['Official Misconduct'] == "Yes") & (exonerations_df['Race'] == "Black")]


In [86]:
race_DNAOM_df[['Crime']].value_counts()

Crime                  
Murder                     108
Sexual Assault              52
Child Sex Abuse              6
Attempted Murder             4
Robbery                      4
Kidnapping                   3
Manslaughter                 3
Accessory to Murder          1
Attempt, Violent             1
Assault                      1
Drug Possession or Sale      1
Burglary/Unlawful Entry      1
Name: count, dtype: int64

In [81]:
raceDNA_count

Race           
Black              349
White              209
Hispanic            62
Native American      4
Other                3
Asian                2
Name: count, dtype: int64

In [87]:
filtered_om = exonerations_df[
    exonerations_df['Race'].isin(['Black', 'White']) &
    exonerations_df['Official Misconduct'].isin(['Yes']) &
    exonerations_df['Sentence']
]

In [89]:
filtered_om.groupby(['Race', 'Official Misconduct', 'Sentence']).size().reset_index(name='Count1')

,Race,Official Misconduct,Sentence,Count1
0,Black,Yes,Death,91
1,Black,Yes,Fine,1
2,Black,Yes,Life,205
3,Black,Yes,Life without parole,162
4,Black,Yes,Not Sentenced,22
5,Black,Yes,Probation,77
6,Black,Yes,Term of Years,794
7,Black,Yes,Unknown,7
8,White,Yes,Community Service,1
9,White,Yes,Death,56
